# Setup

In [ ]:
# !pip install 'crewai[tools]' -q

In [ ]:
# %cd /content/drive/MyDrive/AI Agents/CrewAI/Job Interview Prep Part 2

In [ ]:
# Retrieve API Keys and store them as Environment variables
from google.colab import userdata
openai_api_key = userdata.get('ai_agents_openai')
serper_api_key = userdata.get('serper_api')

import os
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['SERPER_API_KEY'] = serper_api_key

In [ ]:
# import libraries
from crewai import Agent, Crew, Process, Task
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

In [ ]:
# Define the LLM
llm = ChatOpenAI(model_name = "gpt-4o", temperature  = 0.5)

# User Inputs

In [ ]:
# Call for inputs: Interviewer, company, job position, job description:
interviewer = input("Enter the name of the interviewer (if unknown include None): ")
company = input("Enter the name of the company: ")
job_position = input("Enter the job position: ")
job_description = input("Enter the job description: ")

# Research Crew

## Research AI Agent

In [ ]:
# Create the Research AI Agent
research_agent = Agent(
    role = "Research Agent",
    goal = "conduct in-depth research",
    backstory = """ As a Research Specialist, your mission is to uncover detailed and relevant information.
    The ultimate goal is to meet the expectations o the task""",
    llm = llm,
    tools = [ScrapeWebsiteTool(),  SerperDevTool()])

## Research Tasks

In [ ]:
# Define the task about researching the company
research_company_task = Task(
    description = f"""
    Research about {company}, their position in the sector and recent industry trends.
    Focus on recent news and developments from last 12 months""",
    expected_output = f""" A comprehensive report in Markdown with bullet point about {company} and its indutry""",
    agent = research_agent)

In [ ]:
# Research Interviewer Task
research_person_task = Task(
    description = f"Research about {interviewer} with a focus on their professional career",
    expected_output = f"""A comprehensive report in Markdown with bullet point about {interviewer}""",
    agent = research_agent)

## AI Coach + Interview Prep

In [ ]:
# Build the AI interview coach agent
coach_agent = Agent(
    role = "Interview Coach",
    goal = f"I help the user prepare for the job interview for job {job_position}",
    backstory = f"You are an expert in job interviews in companies like {company}",
    llm = llm)

In [ ]:
# Prepare the lsit of questions
define_questions_task = Task(
    description = f"""
    Define a list a of questions to prepare the user for the job {job_position} based on the company {company}, job description {job_description} and interviewer {interviewer}""",
    expected_output = f""" a list of 20 questions to ask the user in Markdown format that cover: Culture and Team fit, Job position Fit, Background and ways of working, Growth Mindset""",
    agent = coach_agent,
    context = [research_company_task, research_person_task])

## Assemble the Crew

In [ ]:
# Assembling the crew
crew = Crew(
    agents = [research_agent, coach_agent],
    tasks = [research_company_task, research_person_task, define_questions_task],
    verbose = True,
    process = Process.sequential)
result = crew.kickoff({"topic": "Write a list of question to prepare for the the interview"})

## Exporting

In [ ]:
# Check the crew outcome
display(Markdown(result.tasks_output[2].raw))

In [ ]:
# Export each of the tasks output
# Exporting the  company report
file_name = f"{company}_report.txt"

# Open the file in write mode and write the content
with open(file_name, "w") as file:
    file.write(result.tasks_output[0].raw)

# Export the interviewer report
file_name = f"{interviewer}_report.txt"

# Open the file in write mode and write the content
with open(file_name, "w") as file:
    file.write(result.tasks_output[1].raw)

# Export the interview questions
file_name = f"{job_position}_questions.txt"

# Open the file in write mode and write the content
with open(file_name, "w") as file:
    file.write(result.tasks_output[2].raw)

# Interview Prep

In [ ]:
# Import the questions from the directory
with open(f"{job_position}_questions.txt", "r") as file:
    questions = file.read()

In [ ]:
# AI Agent that tasks a question
interviewer_agent = Agent(
    role = "Interviewer Agent",
    goal = f"Ask all question from {questions} to the user",
    backstory = f"You are an expert in job interviews in companies like {company}",
    llm = llm)

In [ ]:
# Interview Task
interview_prep_task = Task(
    description = f"Ask a question from {questions} to the user",
    expected_output = f"Ask only one question at a time",
    agent = interviewer_agent,
    human_input = True)

In [ ]:
# Feedback Task
feedback_task = Task(
    description = "Give feedback to the user on its answer",
    expected_output=" Markdown answer of what was good, could be improved and how to take it to the next level",
    agent = coach_agent,
    context = [interview_prep_task],
    human_input = True
    )

## Interview Prep Crew

In [ ]:
# Assemble the interview Prep Crew
crew = Crew(
    agents = [interviewer_agent, coach_agent],
    tasks = [interview_prep_task, feedback_task],
    verbose = True,
    process = Process.sequential)
result = crew.kickoff({"topic": f"Ask me all questions from {questions}"})

In [ ]:
# Assemble the interview Prep Crew
crew = Crew(
    agents = [interviewer_agent, coach_agent],
    tasks = [interview_prep_task, feedback_task],
    verbose = True,
    process = Process.hierarchical,
    manager_llm = ChatOpenAI(model_name = "gpt-4o", temperature  = 0),
    memory = True,
    planning = True)
result = crew.kickoff({"topic": f"Ask me all questions from {questions}"})